In [53]:
!pip install halo
!pip install numerapi
!pip install catboost
!pip install pandas
!pip install pyarrow
!pip install fastparquet

Error in callback <function Halo.__init__.<locals>.clean_up at 0x7fe0436b2440> (for post_run_cell), with arguments args (<ExecutionResult object at 7fe0435859f0, execution_count=53 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fe043586e30, raw_cell="!pip install halo
!pip install numerapi
!pip insta.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/gordontan/Desktop/numerai/july17.ipynb#W0sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

In [55]:
import numpy as np
# from catboost import Pool, CatBoostRegressor
import pandas as pd
import scipy
from halo import Halo
from pathlib import Path
import json
from scipy.stats import skew
import numerapi
import gc
from numerapi import NumerAPI
import time



from utils_plus import *

import pyarrow
import fastparquet


Error in callback <function Halo.__init__.<locals>.clean_up at 0x7fe0436b2440> (for post_run_cell), with arguments args (<ExecutionResult object at 7fe043652860, execution_count=55 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fe0436518d0, raw_cell="import numpy as np
# from catboost import Pool, Ca.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/gordontan/Desktop/numerai/july17.ipynb#W1sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

In [ ]:
!

In [24]:
download_data()

2024-07-17 14:56:48,180 INFO numerapi.utils: starting download
./data/train_int8.parquet: 2.10GB [05:13, 6.69MB/s]                              
2024-07-17 15:02:02,275 INFO numerapi.utils: starting download
./data/validation_int8.parquet: 2.59GB [04:11, 10.3MB/s]                             
2024-07-17 15:06:14,576 INFO numerapi.utils: target file already exists
2024-07-17 15:06:14,576 INFO numerapi.utils: download complete
2024-07-17 15:06:15,456 INFO numerapi.utils: starting download
./data/live_int8.parquet: 6.64MB [00:01, 3.44MB/s]                            


Done!
Error in callback <function Halo.__init__.<locals>.clean_up at 0x7fe0436b2440> (for post_run_cell), with arguments args (<ExecutionResult object at 7fe043194430, execution_count=24 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fe10b4bdde0, raw_cell="download_data()" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/gordontan/Desktop/numerai/july17.ipynb#W3sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

In [58]:
start_time = time.time()

model_name = "july17_1"

with open("./data/features.json", "r") as f:
    feature_metadata = json.load(f)
features = feature_metadata["feature_sets"]["fncv3_features"]
print(features)

# read the training and validation data given the predefined features stored in parquets as pandas DataFrames
training_data, validation_data, live_data = read_prediction_data(features)

# extract feature matrix and target vector used for training
X_train = training_data.filter(like='feature_', axis='columns')
y_train = training_data[TARGET_COL]

# extract feature matrix and target vector used for validation
X_val = validation_data.filter(like='feature_', axis='columns')
y_val = validation_data[TARGET_COL]

X_live = live_data.filter(like='feature_', axis='columns')

# "garbage collection" (gc) gets rid of unused data and frees up memory
gc.collect()


['feature_honoured_observational_balaamite', 'feature_polaroid_vadose_quinze', 'feature_untidy_withdrawn_bargeman', 'feature_genuine_kyphotic_trehala', 'feature_unenthralled_sportful_schoolhouse', 'feature_divulsive_explanatory_ideologue', 'feature_ichthyotic_roofed_yeshiva', 'feature_waggly_outlandish_carbonisation', 'feature_floriated_amish_sprite', 'feature_iconoclastic_parietal_agonist', 'feature_demolished_unfrightened_superpower', 'feature_styloid_subdermal_cytotoxin', 'feature_ironfisted_nonvintage_chlorpromazine', 'feature_litigant_unsizable_rhebok', 'feature_floatiest_quintuplicate_carpentering', 'feature_tuberculate_patelliform_paging', 'feature_cuddlesome_undernamed_incidental', 'feature_loony_zirconic_hoofer', 'feature_indign_tardier_borough', 'feature_fair_papal_vinaigrette', 'feature_attack_unlit_milling', 'feature_froggier_unlearned_underworkman', 'feature_peninsular_pulsatile_vapor', 'feature_midmost_perspiratory_hubert', 'feature_laminable_unspecified_gynoecium', 'feat

ArrowInvalid: No match for FieldRef.Name(target_nomi_v4_20) in era: string
data_type: string
feature_honoured_observational_balaamite: int8
feature_polaroid_vadose_quinze: int8
feature_untidy_withdrawn_bargeman: int8
feature_genuine_kyphotic_trehala: int8
feature_unenthralled_sportful_schoolhouse: int8
feature_divulsive_explanatory_ideologue: int8
feature_ichthyotic_roofed_yeshiva: int8
feature_waggly_outlandish_carbonisation: int8
feature_floriated_amish_sprite: int8
feature_iconoclastic_parietal_agonist: int8
feature_demolished_unfrightened_superpower: int8
feature_styloid_subdermal_cytotoxin: int8
feature_ironfisted_nonvintage_chlorpromazine: int8
feature_litigant_unsizable_rhebok: int8
feature_floatiest_quintuplicate_carpentering: int8
feature_tuberculate_patelliform_paging: int8
feature_cuddlesome_undernamed_incidental: int8
feature_loony_zirconic_hoofer: int8
feature_indign_tardier_borough: int8
feature_fair_papal_vinaigrette: int8
feature_attack_unlit_milling: int8
feature_frogg

Error in callback <function Halo.__init__.<locals>.clean_up at 0x7fe0436b2440> (for post_run_cell), with arguments args (<ExecutionResult object at 7fe043586ef0, execution_count=58 error_before_exec=None error_in_exec=No match for FieldRef.Name(target_nomi_v4_20) in era: string
data_type: string
feature_honoured_observational_balaamite: int8
feature_polaroid_vadose_quinze: int8
feature_untidy_withdrawn_bargeman: int8
feature_genuine_kyphotic_trehala: int8
feature_unenthralled_sportful_schoolhouse: int8
feature_divulsive_explanatory_ideologue: int8
feature_ichthyotic_roofed_yeshiva: int8
feature_waggly_outlandish_carbonisation: int8
feature_floriated_amish_sprite: int8
feature_iconoclastic_parietal_agonist: int8
feature_demolished_unfrightened_superpower: int8
feature_styloid_subdermal_cytotoxin: int8
feature_ironfisted_nonvintage_chlorpromazine: int8
feature_litigant_unsizable_rhebok: int8
feature_floatiest_quintuplicate_carpentering: int8
feature_tuberculate_patelliform_paging: int8
f

TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

In [42]:
num_feature_neutralization = 40  # parameter for feature neutralization used after we make our predictions
params ={"n_estimators": 110,
          "learning_rate": 0.003,
          "task_type":"GPU",
          "depth": 9,
          'l2_leaf_reg': 9}

model = CatBoostRegressor(**params)


spinner.start('Training model')
model.fit(X_train, y_train, early_stopping_rounds = 10, verbose = None)
spinner.succeed()
gc.collect()
spinner.start('Saving model')
model.save_model(f'models/{model_name}.txt')
spinner.succeed()

NameError: name 'CatBoostRegressor' is not defined

Error in callback <function Halo.__init__.<locals>.clean_up at 0x7fe0436b2440> (for post_run_cell), with arguments args (<ExecutionResult object at 7fe10b40c8b0, execution_count=42 error_before_exec=None error_in_exec=name 'CatBoostRegressor' is not defined info=<ExecutionInfo object at 7fe10b40ee30, raw_cell="num_feature_neutralization = 40  # parameter for f.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/gordontan/Desktop/numerai/july17.ipynb#W5sZmlsZQ%3D%3D> result=None>,),kwargs {}:


TypeError: Halo.__init__.<locals>.clean_up() takes 0 positional arguments but 1 was given

In [ ]:
######## Delete belowwhen done using diagnostics tool
live_data.loc[:, f"preds_{model_name}"] = model.predict(X_live)
spinner.succeed()
gc.collect()
neutralize_riskiest_features(training_data, live_data, features, model_name, k=num_feature_neutralization)

print('Exporting Predictions to csv...')
live_data["prediction"] = live_data[f"preds_{model_name}_neutral_riskiest_{num_feature_neutralization}"] \
    .rank(pct=True)
live_data["prediction"].to_csv(f"predictions/{model_name}.csv")



end_time = time.time()
time_elapsed_mins = (end_time - start_time)//60
time_elapsed_secs = (end_time - start_time)%60
print(f"Time elapsed: {time_elapsed_mins} mins {time_elapsed_secs} secs")



print('Done!')